In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import trainer as tn

In [ ]:
data = tn.load_saved_data("./save.txt")

Number of samples

In [ ]:
len(data)

Total time spent practicing:

In [ ]:
total_time = 0
for d in data:
    total_time += d.get("full_history")[-1][2]

print "{}:{}:{}".format(int(total_time/3600),int((total_time%3600)/60),int(total_time%60))

Extracting letter times

In [ ]:
letter_histories = {}
all_letters = []
all_events = []

for d in data:
    full_history_iterator = d.get("full_history").__iter__()
    _, _, prev_time = full_history_iterator.next()[:3]
    batch_datetime = d.get("datetime")
    for event in full_history_iterator:
        if event[0] == "success":
            event_type, letter, time, errored = event
            letter_histories.setdefault(letter,[])
            letter_histories[letter].append(time-prev_time)
            all_letters.append(time-prev_time)
        else:
            event_type, letter, time = event
            errored = None
        all_events.append(dict(
                batch_datetime = batch_datetime,
                letter = letter,
                event_type = event_type,
                time = time,
                dt = time - prev_time,
                errored = errored
            ))
        prev_time = time

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(np.array(letter_histories["e"]))

In [ ]:
letter = "e"
plt.figure(figsize=(12,8))
sns.regplot(np.arange(len(letter_histories[letter])),np.log(np.array(letter_histories[letter])),x_bins=100)

In [ ]:
plt.figure(figsize=(12,8))
sns.regplot(np.arange(len(all_letters)),np.array(all_letters),x_bins=100)

What are my worst letters?

In [ ]:
df = pd.DataFrame(all_events)

In [ ]:
criterion = (df.event_type == "success")
df[criterion].groupby("letter").mean().sort_values("dt")["dt"].plot(kind="barh",figsize=(12,20))

In [ ]:
plot_order = df[criterion].groupby("letter").mean().sort_values("dt").index

In [ ]:
df["log_dt"] = np.log(df["dt"])

In [ ]:
plt.figure(figsize=(16,30))
sns.boxplot(x="dt",y="letter",data=df[criterion], orient="h", order=plot_order, showfliers=False)